# CI-EKS: Covariance Inflation technique (offline)

Here, we use the covariance inflation technique to account for understimated $P^f$ covariance matrix and estimate $R$ matrix. This code is from **Miyoshi 2011**: "The Gaussian approach to adaptive covariance inflation and its implementation with the local ensemble transform Kalman filter", *Monthly Weather Review*.

In [ ]:
# initial counditions and number of iterations
N_iter = 5
lambda_init = 1
sigma2_R_init = 1

# parameters
params = { 'initial_background_state'                 : xb,
           'initial_background_covariance'            : B,
           'initial_multiplicative_inflation'         : lambda_init,
           'initial_observation_noise_variance'       : sigma2_R_init,
           'model_dynamics'                           : f,
           'model_jacobian'                           : jacF,
           'observation_operator'                     : h,
           'observation_jacobian'                     : jacH,
           'observations'                             : Yo,
           'nb_iterations'                            : N_iter,
           'true_state'                               : X_true,
           'state_size'                               : Nx,
           'observation_size'                         : No,
           'temporal_window_size'                     : T,
           'adaptive_parameter'                       : 25000 ### IMPORTANT
         }

# function
res_CI_EKS = CI_EKS(params)

# extract outputs
lambda_CI_EKS = res_CI_EKS['adaptive_multiplicative_inflation']
sigma2_R_CI_EKS = res_CI_EKS['adaptive_observation_noise_variance']
loglik_CI_EKS = res_CI_EKS['loglikelihood']
RMSE_CI_EKS = res_CI_EKS['RMSE']

# LI-EKS: Lag Innovation statistics (online)

Here, we use the lag innovation statistics (difference between consecutive innovations). This code is an extension of the method attached to the publication from **Berry and Sauer 2013**: "Adaptive ensemble Kalman filtering of non-linear systems", *Tellus A*.

In [ ]:
subplot(2,2,1);plot(lambda_CI_EKS)
subplot(2,2,2);plot(sigma2_R_CI_EKS)
subplot(2,2,3);plot(loglik_CI_EKS)
subplot(2,2,4);plot(RMSE_CI_EKS)
print(RMSE_CI_EKS)

In [ ]:
# initial counditions 
lambda_init = 1.93
sigma2_R_init = 1.26

# parameters
params = { 'initial_background_state'                 : xb,
           'initial_background_covariance'            : B,
           'initial_multiplicative_inflation'         : lambda_init,
           'initial_observation_noise_variance'       : sigma2_R_init,
           'model_dynamics'                           : f,
           'model_jacobian'                           : jacF,
           'observation_operator'                     : h,
           'observation_jacobian'                     : jacH,
           'observations'                             : Yo,
           'true_state'                               : X_true,
           'state_size'                               : Nx,
           'observation_size'                         : No,
           'temporal_window_size'                     : T,
           'adaptive_parameter'                       : 20000 ### IMPORTANT
         }

# function
res_CI_EKF = CI_EKF(params)

# extract outputs
lambda_CI_EKF = res_CI_EKF['adaptive_multiplicative_inflation']
sigma2_R_CI_EKF = res_CI_EKF['adaptive_observation_noise_variance']
loglik_CI_EKF = res_CI_EKF['loglikelihood']
RMSE_CI_EKF = res_CI_EKF['RMSE']

In [ ]:
subplot(2,1,1);plot(lambda_CI_EKF);print(lambda_CI_EKF[-1])
subplot(2,1,2);plot(sigma2_R_CI_EKF);print(sigma2_R_CI_EKF[-1])

# LI-EKS: Lag Innovation statistics

Here, we use the lag innovation statistics (difference between consecutive innovations). This code is an extension of the method attached to the publication from **Berry and Sauer 2013**: "Adaptive ensemble Kalman filtering of non-linear systems", *Tellus A*.

In [ ]:
# initial counditions
Q_init = eye(Nx) # identity matrix for initial Q
R_init = eye(No) # identity matrix for initial R

# parameters
params = { 'initial_background_state'                 : xb,
           'initial_background_covariance'            : B,
           'initial_model_noise_covariance'           : Q_init,
           'initial_observation_noise_covariance'     : R_init,
           'model_dynamics'                           : f,
           'model_jacobian'                           : jacF,
           'observation_operator'                     : h,
           'observation_jacobian'                     : jacH,
           'observations'                             : Yo,
           'true_state'                               : X_true,
           'state_size'                               : Nx,
           'observation_size'                         : No,
           'temporal_window_size'                     : T,
           'model_noise_covariance_structure'         : 'full', # Q and R are full
           'inflation_factor'                         : 1,
           'adaptive_parameter'                       : 3000 ### IMPORTANT
         }

# function
res_LI_EKF = LI_EKF(params)

# extract outputs
Q_LI_EKF = res_LI_EKF['LI_model_noise_covariance']
R_LI_EKF = res_LI_EKF['LI_observation_noise_covariance']
loglik_LI_EKF = res_LI_EKF['loglikelihood']
RMSE_LI_EKF = res_LI_EKF['RMSE']

# CI-EnKS VS LI-EnKS (online methods)

Here, we compare the 2 online methods CI-EnKS and LI-EnKS for the estimation of constant $Q$ and $R$ matrices.

In [ ]:
# plot inflation factor
subplot(1,3,1)
line1,=plt.plot(lambda_CI_EKF,'b')
line2,=plt.plot((0,T),(1,1),'--k')
plt.title('Multiplicative inflation estimates')
ylim([0,10])
plt.legend([line1, line2], ['CI-EKF', '$\lambda$=1'])

# plot trace of Q
plt.subplot(1,3,2)
line1,=plt.plot(trace(Q_LI_EKF)/Nx,'g')
line2,=plt.plot((0,T),(trace(Q_true)/Nx,trace(Q_true)/Nx),'--k')
plt.title('Q estimates')
ylim([0,1])
plt.legend([line1, line2], ['LI-EKS', 'True Q'])

# plot trace of R
subplot(1,3,3)
line1,=plt.plot(sigma2_R_CI_EKF,'b')
line2,=plt.plot(trace(R_LI_EKF)/No,'g')
line3,=plt.plot((0,T),(trace(R_true)/No,trace(R_true)/No),'--k')
plt.title('R estimates')
ylim([0,5])
plt.legend([line1, line2], ['CI-EKF', 'LI-EKF', 'True $\sigma^2_o$'])

# print log-likelihood
print('Log-likelihood (CI-EKF): ', loglik_CI_EKF)
print('Log-likelihood (LI-EKF): ', loglik_LI_EKF)
print('Log-likelihood (true): ', loglik_true)

# print Root Mean Square Error
print('RMSE (CI-EKF): ', RMSE_CI_EKF)
print('RMSE (LI-EKF): ', RMSE_LI_EKF)
print('RMSE (true): ', RMSE_true)

# BAYESIAN INFERENCE (ONLINE)

Here, we use the adaptive Bayesian inference for the estimation of the variance parameters of the dynamic error and observation error matrices $Q$ and $R$. This code is attached to the publication from **Stroud et al. 2017**: "A Bayesian adaptive ensemble Kalman filter for sequential state and parameter estimation", *Monthly Weather Review*.

In [ ]:
### APPLY THE BAYESIAN INFERENCE USING EXTENDED KALMAN FILTER (BI-EKF)

sigma2_Q_init = 1
sigma2_R_init = 1

# parameters
params = { 'initial_background_state'                 : xb,
           'initial_background_covariance'            : B,
           'initial_model_noise_variance'             : sigma2_Q_init,
           'initial_observation_noise_variance'       : sigma2_R_init,
           'model_dynamics'                           : f,
           'model_jacobian'                           : jacF,
           'observation_operator'                     : h,
           'observation_jacobian'                     : jacH,
           'observations'                             : Yo,
           'true_state'                               : X_true,
           'state_size'                               : Nx,
           'observation_size'                         : No,
           'temporal_window_size'                     : T,
           'adaptive_parameter'                       : 50
         }

# function
res_BI_EKF = BI_EKF(params)

# extract outputs
sigma2_Q_BI_EKF = res_BI_EKF['adaptive_model_noise_variance']
sigma2_R_BI_EKF = res_BI_EKF['adaptive_observation_noise_variance']
loglik_BI_EKF = res_BI_EKF['loglikelihood']
RMSE_BI_EKF = res_BI_EKF['RMSE']
Xa_BI_EKF = res_BI_EKF['filtered_states']

subplot(1,2,1)
line1,=plt.plot(lambda_BI_EKF,'b')
line2,=plt.plot((0,T),(1,1),'--k')
plt.title('Multiplicative inflation estimates')
ylim([0,5])
plt.legend([line1, line2], ['BI-EKF', '$\lambda$=1'])

subplot(1,2,2)
line1,=plt.plot(sigma2_R_BI_EKF,'b')
line2,=plt.plot((0,T),(trace(R_true)/No,trace(R_true)/No),'--k')
plt.title('R estimates')
ylim([0,5])
plt.legend([line1, line2], ['BI-EKF', 'True $\sigma^2_o$'])